In [1]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

In [2]:
CB = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\18_Rating_CBBBRSME\BRR_CBBBRSME_202206(python).xlsx")


In [3]:
CB_source = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\01_Corporate_Financing\01_Portfolio\20220630\GB Monthly Loan Outstanding 30-Jun-2022 FINAL.xlsx")


In [5]:
CB_source.columns = CB_source.columns.str.upper()
CB_source.columns = CB_source.columns.str.replace(" ", "_")

In [10]:
CB_source1 = CB_source.iloc[np.where((CB_source.M_FINANCING_TYPE_CD=='I')&(CB_source.FACILITY!='RTO')&(CB_source.COMMON_COA_ID==12000)&(CB_source.FACILITY_2!='BOND'))].fillna(0).groupby(['M_CUS_NO','M_FINANCING_TYPE_CD'])[['M_BNM_BALANCE_SUM']].sum().reset_index()


In [ ]:
#patutny rules apply kat CB, bukan CB source

In [11]:
CB_1 = CB.merge(CB_source1, on='M_CUS_NO', how='left', indicator=True)

In [12]:
CB_1.FINAL_RATING = CB_1.FINAL_RATING.str.strip()

CB_1.loc[CB_1.FINAL_RATING == '1', 'RAM'] = 'AAA'
CB_1.loc[CB_1.FINAL_RATING == '2', 'RAM'] = 'AAA'
CB_1.loc[CB_1.FINAL_RATING == '3', 'RAM'] = 'AAA'
CB_1.loc[CB_1.FINAL_RATING == '4', 'RAM'] = 'AA'
CB_1.loc[CB_1.FINAL_RATING == '5', 'RAM'] = 'AA'
CB_1.loc[CB_1.FINAL_RATING == '6', 'RAM'] = 'AA'
CB_1.loc[CB_1.FINAL_RATING == '7', 'RAM'] = 'AA'
CB_1.loc[CB_1.FINAL_RATING == '8', 'RAM'] = 'A'
CB_1.loc[CB_1.FINAL_RATING == '9', 'RAM'] = 'A'
CB_1.loc[CB_1.FINAL_RATING == '10', 'RAM'] = 'A'
CB_1.loc[CB_1.FINAL_RATING == '11', 'RAM'] = 'A'
CB_1.loc[CB_1.FINAL_RATING == '12', 'RAM'] = 'BBB'
CB_1.loc[CB_1.FINAL_RATING == '13', 'RAM'] = 'BBB'
CB_1.loc[CB_1.FINAL_RATING == '14', 'RAM'] = 'BBB'
CB_1.loc[CB_1.FINAL_RATING == '15', 'RAM'] = 'BB'
CB_1.loc[CB_1.FINAL_RATING == '16', 'RAM'] = 'BB'
CB_1.loc[CB_1.FINAL_RATING == '17', 'RAM'] = 'BB'
CB_1.loc[CB_1.FINAL_RATING == '18', 'RAM'] = 'BB'
CB_1.loc[CB_1.FINAL_RATING == '19', 'RAM'] = 'B'
CB_1.loc[CB_1.FINAL_RATING == '20', 'RAM'] = 'B'
CB_1.loc[CB_1.FINAL_RATING == '21', 'RAM'] = 'C'
CB_1.loc[CB_1.FINAL_RATING == '22', 'RAM'] = 'D'
CB_1.loc[CB_1.FINAL_RATING == '23', 'RAM'] = 'D'
CB_1.loc[CB_1.RAM.isna(), 'RAM'] = 'Unrated'


In [21]:
CB_wRP = CB_1.groupby(['RAM'])[['M_BNM_BALANCE_SUM']].sum().reset_index()
CB_wRP

,RAM,M_BNM_BALANCE_SUM
0,A,2.14e+10
1,AA,1.11e+10
2,AAA,1.07e+10
3,B,1.24e+08
4,BB,1.33e+09
5,BBB,9.85e+09
6,C,9.90e+07
7,D,6.30e+09
8,Unrated,6.33e+08


In [22]:
a = sum(CB_wRP.M_BNM_BALANCE_SUM)/10**6
a

61539.003220400016

In [13]:
RPSIA = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\17_Sustainability_ESG high risk sectors\20220331\RPSIA Fikri\RPSIA_Mar22.xlsx", sheet_name='Sheet1')


RPSIA1 = RPSIA.fillna(0)#.groupby(['GCIF'])[['Outstanding']].sum().reset_index()

RPSIA1.GCIF = RPSIA1.GCIF.astype(str)
RPSIA1.GCIF = RPSIA1.GCIF.str.strip()
RPSIA1.GCIF = RPSIA1.GCIF.str.upper()

CB_1.M_CUS_NO = CB_1.M_CUS_NO.astype(str)
CB_1.M_CUS_NO = CB_1.M_CUS_NO.str.strip()
CB_1.M_CUS_NO = CB_1.M_CUS_NO.str.upper()

ex_RPSIA = CB_1.merge(RPSIA1[['GCIF']].drop_duplicates('GCIF').rename(columns={'GCIF':'M_CUS_NO'}), on='M_CUS_NO', how='left', indicator='True')

ex_RPSIA_FINAL = ex_RPSIA.iloc[np.where(ex_RPSIA['True']!='both')]

CB_RPSIA = ex_RPSIA.iloc[np.where(ex_RPSIA['True']=='both')]


CB_2 = ex_RPSIA_FINAL.groupby(['RAM'])[['M_BNM_BALANCE_SUM']].sum().reset_index()

In [16]:
CB_2

,RAM,M_BNM_BALANCE_SUM
0,A,1.13e+10
1,AA,3.82e+09
2,AAA,4.32e+07
3,B,1.24e+08
4,BB,1.33e+09
5,BBB,5.62e+09
6,C,9.90e+07
7,D,4.75e+08
8,Unrated,6.33e+08


In [25]:
RDATE = 20220630

In [26]:
TOTAL_IMPAIRED_FINAL  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(RDATE)[:4]+'\\'+str(RDATE)[:6]+'\working\Total_Impaired_Final_'+str(RDATE)+'.txt',sep = ",", header = 0, low_memory = False)
TOTAL_IMPAIRED_FINAL.columns = TOTAL_IMPAIRED_FINAL.columns.str.upper()
TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER = TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)

In [27]:
TOTAL_IMPAIRED_FINAL1 = TOTAL_IMPAIRED_FINAL.iloc[np.where(TOTAL_IMPAIRED_FINAL.PRODUCT_GROUP=='01.CB')]

In [28]:
TOTAL_IMPAIRED_FINAL2 = TOTAL_IMPAIRED_FINAL1.fillna(0).groupby(['CUSTOMER_ID'])[['OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

In [29]:
TOTAL_IMPAIRED_FINAL3 = TOTAL_IMPAIRED_FINAL2.merge(CB[['M_CUS_NO','NAME','FINAL_RATING']].rename(columns={'M_CUS_NO':'CUSTOMER_ID'}), on='CUSTOMER_ID',how='left',indicator=True)

In [30]:
TOTAL_IMPAIRED_FINAL3.FINAL_RATING = TOTAL_IMPAIRED_FINAL3.FINAL_RATING.str.strip()

TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '1', 'RAM'] = 'AAA'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '2', 'RAM'] = 'AAA'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '3', 'RAM'] = 'AAA'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '4', 'RAM'] = 'AA'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '5', 'RAM'] = 'AA'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '6', 'RAM'] = 'AA'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '7', 'RAM'] = 'AA'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '8', 'RAM'] = 'A'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '9', 'RAM'] = 'A'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '10', 'RAM'] = 'A'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '11', 'RAM'] = 'A'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '12', 'RAM'] = 'BBB'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '13', 'RAM'] = 'BBB'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '14', 'RAM'] = 'BBB'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '15', 'RAM'] = 'BB'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '16', 'RAM'] = 'BB'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '17', 'RAM'] = 'BB'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '18', 'RAM'] = 'BB'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '19', 'RAM'] = 'B'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '20', 'RAM'] = 'B'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '21', 'RAM'] = 'C'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '22', 'RAM'] = 'D'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.FINAL_RATING == '23', 'RAM'] = 'D'
TOTAL_IMPAIRED_FINAL3.loc[TOTAL_IMPAIRED_FINAL3.RAM.isna(), 'RAM'] = 'Unrated'


In [33]:
TOTAL_IMPAIRED_FINAL3.CUSTOMER_ID = TOTAL_IMPAIRED_FINAL3.CUSTOMER_ID.astype(str)
TOTAL_IMPAIRED_FINAL3.CUSTOMER_ID = TOTAL_IMPAIRED_FINAL3.CUSTOMER_ID.str.strip()
TOTAL_IMPAIRED_FINAL3.CUSTOMER_ID = TOTAL_IMPAIRED_FINAL3.CUSTOMER_ID.str.upper()

ex_TOTAL_IMPAIRED_FINAL3 = TOTAL_IMPAIRED_FINAL3.merge(RPSIA1[['GCIF']].drop_duplicates('GCIF').rename(columns={'GCIF':'CUSTOMER_ID'}), on='CUSTOMER_ID', how='left', indicator='True')



ex_TOTAL_IMPAIRED_FINAL3_FINAL = ex_TOTAL_IMPAIRED_FINAL3.iloc[np.where(ex_TOTAL_IMPAIRED_FINAL3['True']!='both')]

In [41]:
IF_RPSIA = ex_TOTAL_IMPAIRED_FINAL3.iloc[np.where(ex_TOTAL_IMPAIRED_FINAL3['True']=='both')]
IF_RPSIA

,CUSTOMER_ID,OUTSTANDING_AFTER_EIR_UWI,NAME,FINAL_RATING,_merge,RAM,True
16,19492758,3.69e+08,TRURICH RESOURCES SDN BHD,23,both,D,both


In [39]:
TOTAL_IMPAIRED_FINAL4 = ex_TOTAL_IMPAIRED_FINAL3_FINAL.groupby(['RAM'])[['OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

In [43]:
writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\202208_RAM\Corp & SME Q15 & Q16\RAM_CB - jun 22 (exclude RPSIA).xlsx',engine='xlsxwriter')
CB_RPSIA.to_excel(writer2,sheet_name='CB - RPSIA' ,index = False)
CB_1.to_excel(writer2,sheet_name='CB - base' ,index = False)
CB_2.to_excel(writer2,sheet_name='CB' ,index = False)
TOTAL_IMPAIRED_FINAL4.to_excel(writer2,sheet_name='CB IF' ,index = False)
TOTAL_IMPAIRED_FINAL3.to_excel(writer2,sheet_name='CB IF - base' ,index = False)
IF_RPSIA.to_excel(writer2,sheet_name='CB IF - RPSIA' ,index = False)
writer2.save()